<a href="https://colab.research.google.com/github/mehakhafeez/lab-abstractive-question-answering/blob/main/lab-abstractive-question-answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [2]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


wikipedia_snippets_streamed.py:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

The repository for vblagoje/wikipedia_snippets_streamed contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/vblagoje/wikipedia_snippets_streamed.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [3]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [4]:
# filter only documents with History as section_title - Replace None with your code
history = None

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [10]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []

history = wiki_data.filter(lambda x: 'History' in str(x['section_title']))

for d in tqdm(history, total=total_doc_count):
    # extract the fields we need - article, section, and passage
    if 'History' in d['section_title']:  # Check if 'History' is in the section title
        docs.append({
            'article_title': d['article_title'],
            'section_title': d['section_title'],
            'passage_text': d['passage_text']
        })
    # increase the counter on every iteration
    counter += 1
    if counter >= total_doc_count:
        break

  0%|          | 0/50000 [00:00<?, ?it/s]

In [11]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Taupo District,History,was not until the 1950s that the region starte...
1,Sutarfeni,History & Western asian analogues,Sutarfeni History strand-like pheni were Phena...
2,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
3,Teufelsmoor,History & Situation today,"made to preserve the original landscape, altho..."
4,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [14]:
import os
from pinecone import Pinecone

from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_API_KEY=PINECONE_API_KEY
# configure client
pc = Pinecone(api_key=PINECONE_API_KEY)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [15]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [16]:
index_name = "abstractive-question-answering" #give your index a meaningful name

In [17]:
import time

# check if index already exists (it shouldn't if this is first time)
# Create the index
pc.create_index(name=index_name, metric='cosine', dimension=768, spec=spec)

# Check if the index exists
index = pc.Index(index_name) #initialize the index, and insure the stats are all zeros

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [18]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')
retriever.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [20]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    batch = df.iloc[i:i + batch_size]

    # Generate embeddings for the passage texts
    embeddings = retriever.encode(batch['passage_text'].tolist(), convert_to_tensor=True)

    # Prepare metadata
    meta_data = [
        {'article_title': row['article_title'], 'section_title': row['section_title'], 'passage_text': row['passage_text']}
        for _, row in batch.iterrows()
    ]

    # Prepare IDs (use article title as unique ID) - Replace article title with ASCII representation
    ids = [str(id).encode('ascii', 'ignore').decode('ascii') for id in batch['article_title'].tolist()]

    # Upsert embeddings to Pinecone
    index.upsert(vectors=list(zip(ids, embeddings.tolist(), meta_data)))

index.describe_index_stats()

  0%|          | 0/782 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 18026}},
 'total_vector_count': 18026}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [21]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [28]:
def query_pinecone(query, top_k=5):
    # generate embeddings for the query
    xq = query_embedding = retriever.encode([query], convert_to_tensor=True)
    # search pinecone index for context passage with the answer
    # Pass query_embedding as a keyword argument named 'vector'
    xc = result = index.query(vector=query_embedding.tolist(), top_k=top_k, include_metadata=True)
    return xc

In [29]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = f"question: {query} context: {' '.join(context)}" #Corrected line
    return context #Corrected line

def generate_answer(query): #Corrected indentation
    # Format the query and context
    context = query_pinecone(query, top_k=5)
    formatted_query = format_query(query, context['matches']) # Access using ['matches']

    # Tokenize the formatted query
    inputs = tokenizer([formatted_query], max_length=1024, return_tensors="pt").to(device)

    # Generate the answer
    answer_ids = generator.generate(inputs["input_ids"], num_beams=4, min_length=20, max_length=100)

    # Decode the answer
    answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)

    return answer

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [30]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': 'Electric power system',
              'metadata': {'article_title': 'Electric power system',
                           'passage_text': 'Electric power system History In '
                                           '1881, two electricians built the '
                                           "world's first power system at "
                                           'Godalming in England. It was '
                                           'powered by two waterwheels and '
                                           'produced an alternating current '
                                           'that in turn supplied seven '
                                           'Siemens arc lamps at 250 volts and '
                                           '34 incandescent lamps at 40 volts. '
                                           'However, supply to the lamps was '
                                           'intermittent and in 1882 Thomas '
                             

In [32]:
from pprint import pprint

In [33]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [34]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [35]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [36]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The first wireless message was sent by a man named Guglielmo Marconi in '
 '1896. He used a dipole antenna to transmit a message to a man named Heinrich '
 'Hertz')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [37]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

audio distance records, and were heard as far west as Hawaii. They were also received in Paris, France, which marked the first transmission of speech across the Atlantic.
With the entrance of the United States into World War I in April 1917 the federal government took over full control of the radio industry, and it became illegal for civilians to possess an operational radio receiver. However NAA continued to operate during the conflict. In addition to time signals and weather reports, it also broadcast news summaries received by troops on land and aboard ships in the Atlantic. Effective April 15, 1919
---
Message from space (science fiction) For other uses, see Message from Space (disambiguation).
"Message from space" is a type of "first contact" theme in science fiction . Stories of this type involve receiving an interstellar message which reveals the existence of other intelligent life in the universe. History An early short story, A Message from Space (Joseph Schlossel, Weird Tales

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [38]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is not a virus, it is a bacterium. It is a virus that infects '
 'bacteria. It is not a bacterium, it is a bacterium that infect')


In [39]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild animals into areas of human activity. An example of this is the outbreak of Nipah virus in peninsular Malaysia in 1999, when intensive pig farming began on the habitat of infected fruit bats. Unidentified infection of the pigs amplified the force of infection, eventually transmitting the virus
---
days of the ancient Greeks. After an outbreak in Edinburgh in the 1840s, relapsing fever was given its name, but the etiology of the disease was not better understood for a decade. Physician David Livingstone is credited with the first account in 1857 of a malady associated with the bite of soft ticks in Angola and Mozambique. In 1873, Otto Obermeier 

Let’s finish with a final few questions.

In [40]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The war of currents is a term used to refer to a series of events in the '
 'history of the world that occurred in the early 20th century. The most '
 'famous of these events was the')


In [41]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong, who walked on the '
 'moon in 1969. He was the first man to walk on the moon, and the first man to '
 'walk on')


In [42]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


In [43]:
query = "when was the first electric power system built?"
answer = generate_answer(query)
print(answer)

('Electricity was first used in the 19th century. The first electric power '
 'system built was a steam engine.')
None


In [44]:
query = "What is a sonic boom?"
answer = generate_answer(query)
print(answer)

('What is a sonic boom? What is a sonic boom? What is a sonic boom? What is a '
 'sonic boom?')
None


As we can see, the model can generate some decent answers.

#### Add a few more questions